```python 
from transformers import AutoTokenizer, TrainingArguments, Trainer, AutoModel
import numpy as np
import torch
from datasets import load_dataset
import torch.nn as nn
import os
from typing import List
from tqdm import tqdm


os.environ["CUDA_VISIBLE_DEVICES"] = "1" ## Setup CUDA GPU 1


class BERTIntentClassification(nn.Module):
    
    
    def __init__(self, model_name="bert-base-uncased", num_classes=10, dropout_rate=0.1, cache_dir = "huggingface"):
        super(BERTIntentClassification, self).__init__()
        self.bert = AutoModel.from_pretrained(model_name, cache_dir = cache_dir)
        # Get BERT hidden size
        hidden_size = self.bert.config.hidden_size
        self.ffnn = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.LayerNorm(hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_size, num_classes)
        )

    
    def freeze_bert(self):
        for param in self.bert.parameters():
            param.requires_grad = False


    def get_pooling(self, hidden_state, attention_mask):
        """
        Get mean pooled representation from BERT hidden states
        Args:
            hidden_state: BERT output containing hidden states
        Returns:
            pooled_output: Mean pooled representation of the sequence
        """
        # Get last hidden state
        last_hidden_state = hidden_state.last_hidden_state  # Shape: [batch_size, seq_len, hidden_size]
        
        if attention_mask is not None:
            # Expand attention mask to match hidden state dimensions
            attention_mask = attention_mask.unsqueeze(-1)  # [batch_size, seq_len, 1]
            
            # Mask out padding tokens
            masked_hidden = last_hidden_state * attention_mask
            
            # Calculate mean (sum / number of actual tokens)
            sum_hidden = torch.sum(masked_hidden, dim=1)  # [batch_size, hidden_size]
            count_tokens = torch.sum(attention_mask, dim=1)  # [batch_size, 1]
            pooled_output = sum_hidden / count_tokens
        else:
            # If no attention mask, simply take mean of all tokens
            pooled_output = torch.mean(last_hidden_state, dim=1)
        
        return pooled_output
        
    
    def forward(self, input_ids, attention_mask, **kwargs):
        """
        Forward pass of the model
        Args:
            input_ids: Input token IDs
            attention_mask: Attention mask for padding
        Returns:
            logits: Raw logits for each class
        """
        # Get BERT hidden states
        hidden_state = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
        )
        
        # Get pooled representation
        hidden_state_pooling = self.get_pooling(hidden_state=hidden_state, attention_mask=attention_mask)
        
        # Pass through FFNN classifier
        logits = self.ffnn(hidden_state_pooling)
        
        return logits



class TrainerCustom(Trainer):

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        """
        How the loss is computed by Trainer. By default, all models return the loss in the first element.

        Subclass and override for custom behavior.
        """
        if "labels" in inputs:
            labels = inputs.pop("labels")
        else:
            labels = None
        
        # Sử dụng nn.CrossEntropyLoss() thay vì nn.CrossEntropy
        cross_entropy_loss = nn.CrossEntropyLoss()
        
        # Chạy mô hình và nhận đầu ra (logits)
        outputs = model(**inputs)
        
        # Đảm bảo lấy logits từ outputs (mô hình trả về tuple, lấy phần tử đầu tiên là logits)
        logits = outputs
        
        # Tính toán loss
        loss = cross_entropy_loss(logits, labels)
        
        # Trả về loss và outputs nếu cần
        return (loss, outputs) if return_outputs else loss


# Bước 1: Tải dữ liệu
# Sử dụng dataset sẵn có từ Hugging Face hoặc tải từ file cục bộ
dataset = load_dataset("imdb", cache_dir = "huggingface")  # Ví dụ: Dữ liệu IMDB để phân loại sentiment
# Thay thế trường 'text' thành 'input_ids' trong train_dataset và test_dataset
def preprocess_dataset(dataset):
    return dataset.map(lambda example: {
            "input_ids": example['text'],
            "label": example['label']
        }, 
        remove_columns=["text"],
        num_proc=4  # Sử dụng 4 tiến trình song song để xử lý nhanh hơn
    )

train_dataset = preprocess_dataset(dataset["train"])
test_dataset = preprocess_dataset(dataset["test"])


# Bước 2: Chuẩn bị tokenizer và token hóa dữ liệu
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir = "huggingface")
model = BERTIntentClassification(
    model_name=model_name,
    num_classes=2
)
model.freeze_bert() # Froze Layer BERT
max_seq_length = 512


def collate_fn(features):
    inputs = []
    labels = []
    for element in features:
        inputs.append(element.get("input_ids"))
        labels.append(element.get("label"))
    
    labels = torch.tensor(labels, dtype=torch.long)
    
    token_inputs = tokenizer(
        inputs,
        add_special_tokens=True,
        truncation=True,
        padding=True,
        max_length=max_seq_length,
        return_overflowing_tokens=False,
        return_length=False,
        return_tensors="pt",
    )
    token_inputs.update({
        "labels": labels,
    })
    return token_inputs

# Bước 6: Cài đặt tham số huấn luyện
training_args = TrainingArguments(
    output_dir="./results",          # Thư mục lưu kết quả
    eval_strategy="epoch",    # Đánh giá sau mỗi epoch
    learning_rate=2e-4,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    num_train_epochs=50,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=None,
    logging_strategy = "epoch",
    save_strategy="epoch",          # Lưu trọng số sau mỗi epoch
    save_total_limit=3,
)

# Bước 7: Tạo Trainer
trainer = TrainerCustom(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator = collate_fn,
)

# Bước 8: Huấn luyện
trainer.train()

# Bước 9: Đánh giá trên tập kiểm tra
trainer.evaluate()
```
